In [70]:
import sys
sys.path.append('../')
import functions
exec(open('./functions.py').read())

demoji.download_codes()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\home\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\home\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
... OK (Got response in 0.62 seconds)
Writing emoji data to C:\Users\home\.demoji\codes.json ...
... OK
... OK (Got response in 0.51 seconds)
Writing emoji data to C:\Users\home\.demoji\codes.json ...
... OK


In [10]:
#load labelled data (generated in file 2.preprocessing)
data = pickle.load(open('eng_df_labelled.pkl', "rb"))

In [12]:
#tag each tweet with unique id
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(data.split)]
documents

ry', 'asos'], tags=[853]),
 TaggedDocument(words=['I', 've', 'live', 'sweatpant', 'jammie', 'long', 'I', 've', 'go', 'asos', 'buy', 'bit', 'world', 'open', 'work', 'zero', 'idea', 'style', 'anymore'], tags=[854]),
 TaggedDocument(words=['amazing', 'friend'], tags=[855]),
 TaggedDocument(words=['it', '’', 'birth', 'week', 'cry', 'face', 'face', 'blowing', 'kiss'], tags=[856]),
 TaggedDocument(words=['wing', 'way', 'asos', 'today', 'wear', 'black', 'legging', 'shirt', 'laptop'], tags=[857]),
 TaggedDocument(words=['nordstrom', 'asos', 'want', 'build', 'powerful', 'partnership', 'topshop', 'plan'], tags=[858]),
 TaggedDocument(words=['joke', 'parcel'], tags=[859]),
 TaggedDocument(words=['spend', 'entire', 'paycheck', 'asos', 'regularly', 'skull'], tags=[860]),
 TaggedDocument(words=['apparently', 'converse', 'want', 'asos', 'shipping', 'restriction', 'smh'], tags=[861]),
 TaggedDocument(words=['would', 'n’t', 'add', 'wardrobe', 'need', 'basic'], tags=[862]),
 TaggedDocument(words=['I', '

In [26]:
from collections import Counter

In [88]:
#create a flat list of all the words
words = [item for sublist in list(data.split) for item in sublist]

In [89]:
#explore the absolute frequencies to get the idea of minimum count for the future Doc2Vec Model
c = Counter(words)
counts = pd.DataFrame.from_dict(c, orient='index', columns = ['count']).reset_index()
print(counts.describe())
counts['count'].quantile([0.90, 0.95, 0.99])

              count
count  32950.000000
mean      35.424370
std      417.319219
min        1.000000
25%        1.000000
50%        2.000000
75%        7.000000
max    42815.000000


0.90     29.00
0.95     80.00
0.99    606.02
Name: count, dtype: float64

90% of words have less than 29 appearances in the whole dataset. So that we will take into account only those which have more than 29.

In [48]:
#build the model
Doc2Vec_model = Doc2Vec(documents, vector_size=5, window=2, min_count=29, workers=4)

In [49]:
#create a DataFrame with a tweet and a vector for each tweet
corpus_d2v=pd.DataFrame([Doc2Vec_model.infer_vector(doc) for doc in data.split])

In [74]:
#print the vocabulary: all the words are considered in the model
print(Doc2Vec_model.wv.key_to_index)

': 2099, 'idaho': 2100, 'investigate': 2101, 'description': 2102, 'adam': 2103, 'paddy': 2104, 'obscd': 2105, 'secret': 2106, 'walmart': 2107, 'personally': 2108, 'practice': 2109, 'reward': 2110, 'asc': 2111, 'weekday': 2112, 'dangerous': 2113, 'ping': 2114, 'outta': 2115, 'responsible': 2116, 'brunch': 2117, 'excellent': 2118, 'chinese': 2119, 'master': 2120, 'thrifte': 2121, 'achieve': 2122, 'playing': 2123, 'earn': 2124, 'awe': 2125, 'the': 2126, 'quid': 2127, 'obsc': 2128, 'reorder': 2129, 'sorted': 2130, 'majority': 2131, 'bell': 2132, 'mfs': 2133, 'ego': 2134, 'bridesmaid': 2135, 'gutte': 2136, 'def': 2137, 'soul': 2138, 'letter': 2139, 'modern': 2140, 'rent': 2141, 'screenshot': 2142, 'possibly': 2143, 'password': 2144, 'corporate': 2145, 'toy': 2146, 'bottom': 2147, 'jail': 2148, 'sized': 2149, 'rubbish': 2150, 'john': 2151, 'slowfashion': 2152, 'dsnt': 2153, 'meme': 2154, 'tooth': 2155, 'stretch': 2156, 'certainly': 2157, 'expedia': 2158, 'capitalism': 2159, 'protect': 2160, 

In [52]:
print(corpus_d2v.head())
len(corpus_d2v)

          0         1         2         3         4
0 -0.015905  0.191896  0.311980 -0.082201 -0.054656
1  0.094732 -0.196510  0.232367 -0.184807 -0.250304
2 -0.028408  0.077553  0.232172 -0.400177  0.024043
3 -0.025499 -0.010442 -0.289881 -0.029361 -0.232676
4 -0.092368  0.046109  0.195366 -0.112181 -0.068188


109987

In [83]:
#merge embeddings and initial dataset, keep only embeddings and label(sentiment)
data_represented_word2vec = data.merge(corpus_d2v, left_index =True, right_index=True)[['compound', 0, 1, 2,3,4]]

In [84]:
data_represented_word2vec

,compound,0,1,2,3,4
0,0.0000,-0.015905,0.191896,0.311980,-0.082201,-0.054656
1,-0.9060,0.094732,-0.196510,0.232367,-0.184807,-0.250304
2,0.6249,-0.028408,0.077553,0.232172,-0.400177,0.024043
3,-0.4767,-0.025499,-0.010442,-0.289881,-0.029361,-0.232676
4,0.0000,-0.092368,0.046109,0.195366,-0.112181,-0.068188
...,...,...,...,...,...,...
109982,0.0000,-0.163756,-0.038913,0.057557,-0.313182,0.062956
109983,0.0000,-0.072948,-0.031609,-0.081971,-0.050795,-0.041536
109984,0.7351,-0.322610,-0.180934,-0.136625,-0.398735,-0.048561
109985,0.1867,-0.058633,0.367701,-0.022004,-0.125063,-0.020028


In [85]:
#save the dataset
pickle.dump(data_represented_word2vec, open(r'.\eng_df_represented_doc2vec.pkl', 'wb'))

In [86]:
# Save trained doc2vec model
Doc2Vec_model.save("eng_doc2vec.model")
## Load saved doc2vec model
#model= Doc2Vec_model.load("eng_doc2vec.model")